In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
from cot import Collection

# 1) Dataset loading and selecting a random sample
collection = Collection(["med_qa"], verbose=False)
# collection = collection.select(split="validation")

# 2) Language Model generates chains of thought and then extracts answers
config={
    "instruction_keys": ['qa-01'], # "Answer the following question through step-by-step reasoning."
    "cot_trigger_keys": ['kojima-01'], # "Answer: Let's think step by step."
    "answer_extraction_keys": ['kojima-A-D'], # "Therefore, among A through D, the answer is"
    "api_service": "huggingface_hub",
    "engine": "google/flan-t5-xl",
    "warn": False,
    "verbose": False,
}
collection.generate(config=config)

# 3) Performance evaluation
collection.evaluate()

Loading med_qa...
{}


/home/kon/CoT/ThoughtSource/libs/cot/cot/evaluate.py:84: UserWarning: Your answer could not be extracted, please add your sequence to the list of personalized answers sequences.
                sequence: {pred}
                In the file: libs/cot/cot/evaluate.py under the function clean()
  warnings.warn(


{'accuracy': {'None_lievin-02_kojima-A-D': 0.19324430479183033, 'None_lievin-10_kojima-A-D': 0.19402985074626866, 'None_lievin-03_kojima-A-D': 0.19638648860958366, 'None_lievin-01_kojima-A-D': 0.19088766692851533, 'None_kojima-01_kojima-A-D': 0.192458758837392}}
{}


In [13]:
import re

txt = "Therefore, among A through E, the answer is (d)."
expected_answer_location = r"\s?[\(\{\[]?(.)[\)\}\]]?\.?"
starting_sequence = r"[Aa]nswer:?\s?(?:is)?\s?" + expected_answer_location
in_one = r"[Aa]nswer:?\s?(?:is)?\s?\s?[\(\{\[]?(.)[\)\}\]]?\.?"
x = re.search(starting_sequence, txt)
print(x)
print(x.group(1))

<re.Match object; span=(34, 48), match='answer is (d).'>
d


In [19]:
import re

txt = "Therefore, among A through E, the answer is (d)."

starting_sequence = r'answer:?\s(?:is)?\s?\s?[\(\{\[]?(A|B|C|D|E|F|G)[\)\}\]]?\.?'

x = re.search(starting_sequence, txt, re.MULTILINE | re.IGNORECASE)
print(x)
print(x.group(1))

<re.Match object; span=(34, 48), match='answer is (d).'>
d


In [20]:
starting_sequence

'answer:?\\s(?:is)?\\s?\\s?[\\(\\{\\[]?(A|B|C|D|E|F|G)[\\)\\}\\]]?\\.?'

In [15]:
r'answer:?\\s(?:is)?\\s?\\s?[\\(\\{\\[]?(A|B|C|D|E|F|G)[\\)\\}\\]]?\\.?'

'answer:?\\\\s(?:is)?\\\\s?\\\\s?[\\\\(\\\\{\\\\[]?(A|B|C|D|E|F|G)[\\\\)\\\\}\\\\]]?\\\\.?'

In [14]:
starting_sequence

'[Aa]nswer:?\\s?(?:is)?\\s?\\s?[\\(\\{\\[]?(.)[\\)\\}\\]]?\\.?'

In [27]:
x.group(1)

'D'

In [3]:
r"[Aa]nswer:?\s?(?:is)?\s?\s?[\(\{\[]?(.)[\)\}\]]?\.?"

'[Aa]nswer:?\\s?(?:is)?\\s?\\s?[\\(\\{\\[]?(.)[\\)\\}\\]]?\\.?'

In [5]:
r"\a"

'\\a'

In [7]:
R"\a"

'\\a'

In [39]:
import re

regex = r"[Aa]nswer:?\s?(?:is)?\s?" + r"\s?[\(\{\[]?(.)[\)\}\]]?\.?"

test_str = "Therefore, among A through E, the answer is D."

matches = re.search(regex, test_str, re.MULTILINE)

if matches:
    print ("Match was found at {start}-{end}: {match}".format(start = matches.start(), end = matches.end(), match = matches.group()))
    
    for groupNum in range(0, len(matches.groups())):
        groupNum = groupNum + 1
        
        print ("Group {groupNum} found at {start}-{end}: {group}".format(groupNum = groupNum, start = matches.start(groupNum), end = matches.end(groupNum), group = matches.group(groupNum)))

Match was found at 34-46: answer is D.
Group 1 found at 44-45: D


In [38]:
matches.group(1)

'D'

In [5]:
collection.dump("commonsense_qa_validation.json")

In [7]:
collection["commonsense_qa"]["validation"][0]

{'id': '7b95825a19d6930d6aed35c7c57a2d82',
 'ref_id': '',
 'question': "James couldn't get comfortable.  There was too much dirt.  He needed to clean out what?",
 'type': 'multiplechoice',
 'choices': ['ground', 'subway', 'bank', 'bed', 'street'],
 'context': '',
 'cot': ['Bed is a piece of furniture for sleep or rest, typically a framework with a mattress.',
  "James couldn't get comfortable.",
  'There was too much dirt.',
  'He needed to clean out bed.',
  'James could have found the ground with too much dirt but it’s not possible to clean the whole ground.',
  'Subways were under several feet of dirt, james always found it claustrophobic and he hated using it so, it was not what he needed to clean out.',
  'Bank is not a private property that could be needed for james to clean up.',
  'Cleaning of streets is a responsibility of the administrative body of a city or a town and not for james to worry about.'],
 'answer': ['bed'],
 'generated_cot': [{'id': '887',
   'fragments_version'

In [3]:
for  item in collection["commonsense_qa"]["validation"]:
    print(
        item['generated_cot'][0]['answers'][0]['correct_answer'])

True
True
True
False
True
True
True
True
True
False
